In [1]:
# Andmete sisselugemine
import pandas as pd
import numpy as np

happy_2015_df = pd.read_csv("2015.csv")
happy_2016_df = pd.read_csv("2016.csv")
happy_2017_df = pd.read_csv("2017.csv")
happy_2018_df = pd.read_csv("2018.csv")
happy_2019_df = pd.read_csv("2019.csv")

# Lisame tunnuse "year"
happy_2015_df["year"] = 2015
happy_2016_df["year"] = 2016
happy_2017_df["year"] = 2017
happy_2018_df["year"] = 2018
happy_2019_df["year"] = 2019

# Valime need veerud, mis kõigil olemas on ja paneme need samasse järjekorda.
happy_2015 = happy_2015_df.iloc[:, [0, 2, 3, 5, 7, 8, 9, 10, 12]]
happy_2015.columns = ["country", "rank", "score", "economy", "health", "freedom", "trust", "generosity", "year"]
happy_2016 = happy_2016_df.iloc[:, [0, 2, 3, 6, 8, 9, 10, 11, 13]]
happy_2016.columns = ["country", "rank", "score", "economy", "health", "freedom", "trust", "generosity", "year"]
happy_2017 = happy_2017_df.iloc[:, [0, 1, 2, 5, 7, 8, 9, 10, 12]]
happy_2017.columns = ["country", "rank", "score", "economy", "health", "freedom", "generosity", "trust", "year"]
happy_2017 = happy_2017[["country", "rank", "score", "economy", "health", "freedom", "trust", "generosity", "year"]]
happy_2018 = happy_2018_df.iloc[:, [0, 1, 2, 3, 5, 6, 7, 8, 9]]
happy_2018.columns = ["rank", "country", "score", "economy", "health", "freedom", "generosity", "trust", "year"]
happy_2018 = happy_2018[["country", "rank", "score", "economy", "health", "freedom", "trust", "generosity", "year"]]
happy_2019 = happy_2019_df.iloc[:, [0, 1, 2, 3, 5, 6, 7, 8, 9]]
happy_2019.columns = ["rank", "country", "score", "economy", "health", "freedom", "generosity", "trust", "year"]
happy_2019 = happy_2019[["country", "rank", "score", "economy", "health", "freedom", "trust", "generosity", "year"]]

# Ühendame andmestikud
happy = pd.concat([happy_2015, happy_2016, happy_2017, happy_2018, happy_2019], axis=0)
happy = happy.reset_index(drop=True) # Ajame indeksid paika

# Teeme sama nimega need, mis tegelikult on kõigis olemas
happy["country"] = happy["country"].replace("Hong Kong S.A.R., China", "Hong Kong")
happy["country"] = happy["country"].replace("Northern Cyprus", "North Cyprus")
happy["country"] = happy["country"].replace("Taiwan Province of China", "Taiwan")
happy["country"] = happy["country"].replace("Trinidad and Tobago", "Trinidad & Tobago")
# Muudame ka kongosid, et teiste andmestikega klapiks
happy["country"] = happy["country"].replace("Congo (Kinshasa)", "Democratic Republic of the Congo")
happy["country"] = happy["country"].replace("Congo (Brazzaville)", "Republic of the Congo")

# Vaatame uuesti, mis on puudulikud ja eemaldame need andmestikust (Somalia???)
loendus = happy.groupby('country').count()
loendus = loendus["rank"]
loendus = loendus[loendus < 5]
puudulikud = loendus.index.tolist()
happy = happy[~happy['country'].isin(puudulikud)]

# teeme AÜE korda
happy[happy['trust'].isnull()] # 2018 AÜE trust puudu, asendame 2017 ja 2019 keskmisega
kesk = (float(happy.loc[(happy["country"] == "United Arab Emirates") & (happy["year"] == 2017),"trust"]) + float(happy.loc[(happy["country"] == "United Arab Emirates") & (happy["year"] == 2019),"trust"]))/2
happy.loc[(happy["country"] == "United Arab Emirates") & (happy["year"] == 2018),"trust"] = kesk

happy

,country,rank,score,economy,health,freedom,trust,generosity,year
0,Switzerland,1,7.587,1.39651,0.94143,0.66557,0.41978,0.29678,2015
1,Iceland,2,7.561,1.30232,0.94784,0.62877,0.14145,0.43630,2015
2,Denmark,3,7.527,1.32548,0.87464,0.64938,0.48357,0.34139,2015
3,Norway,4,7.522,1.45900,0.88521,0.66973,0.36503,0.34699,2015
4,Canada,5,7.427,1.32629,0.90563,0.63297,0.32957,0.45811,2015
...,...,...,...,...,...,...,...,...,...
775,Malawi,150,3.410,0.19100,0.49500,0.44300,0.08900,0.21800,2019
776,Yemen,151,3.380,0.28700,0.46300,0.14300,0.07700,0.10800,2019
777,Rwanda,152,3.334,0.35900,0.61400,0.55500,0.41100,0.21700,2019
778,Tanzania,153,3.231,0.47600,0.49900,0.41700,0.14700,0.27600,2019


In [2]:
GDP_df = pd.read_excel("ProjectedRealPerCapitaGDPValues.xls" , sheet_name='Projected Real GDP Per Capita')

riigid = happy["country"].unique().tolist()
GDP = GDP_df[GDP_df['Country'].isin(riigid)] # Valime välja vaid happiness reportis esinevad riigid
GDP = GDP.iloc[:,[0, 5,6,7,8,9]]

# Uurime mis välja jäid
riigid2 = GDP["Country"].tolist()
valja = set(riigid).difference(set(riigid2))
valja

# Vaatame käsitsi need riigid GDP excelist üle ja vajadusel muudame nimesid

GDP_df["Country"] = GDP_df["Country"].replace("Côte d'Ivoire", "Ivory Coast")
GDP_df["Country"] = GDP_df["Country"].replace("Korea", "South Korea")
GDP_df["Country"] = GDP_df["Country"].replace("Trinidad and Tobago", "Trinidad & Tobago")

# Kaasame väljajäänud riigid
GDP = GDP_df[GDP_df['Country'].isin(riigid)] 
GDP = GDP.iloc[:,[0, 5,6,7,8,9]]

GDP

,Country,2015,2016,2017,2018,2019
3,Canada,51354.109375,51692.875000,52878.199219,53484.242188,53825.738281
4,United States,52005.167969,52525.148438,53296.578125,54421.187500,55380.246094
7,Mexico,10047.209961,10218.336914,10309.086914,10399.106445,10343.576172
13,Costa Rica,9283.854492,9562.051758,9760.123047,9905.846680,10053.239258
15,Dominican Republic,6578.867676,6929.162109,7536.377930,7979.916992,8336.320312
...,...,...,...,...,...,...
209,Republic of the Congo,3073.526367,2927.553467,2778.249023,2747.220703,2773.413574
210,Rwanda,730.225586,758.713989,784.616150,832.652344,871.358826
215,Tanzania,853.806335,887.457886,921.888000,943.584595,974.751526
216,Uganda,659.463257,669.037720,672.916260,691.467224,709.582275


In [3]:
# Ühendame GDP ja happy andmestikud
happy2 = happy[happy['country'].isin(GDP["Country"].tolist())] 
happy2["GDP"] = 0
for i in range(0, happy2.shape[0]):
    aasta = happy2.iloc[i, 8]
    riik = happy2.iloc[i, 0]
    aasta_andmed = GDP[["Country", aasta]]
    rida = aasta_andmed[aasta_andmed["Country"] == riik]
    happy2.iloc[i, 9] = float(rida[aasta])
happy2

<ipython-input-3-70ee63ae76bf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  happy2["GDP"] = 0
C:\Users\Laura Raudsepp\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,country,rank,score,economy,health,freedom,trust,generosity,year,GDP
0,Switzerland,1,7.587,1.39651,0.94143,0.66557,0.41978,0.29678,2015,78065.890625
1,Iceland,2,7.561,1.30232,0.94784,0.62877,0.14145,0.43630,2015,47247.511719
2,Denmark,3,7.527,1.32548,0.87464,0.64938,0.48357,0.34139,2015,61505.738281
3,Norway,4,7.522,1.45900,0.88521,0.66973,0.36503,0.34699,2015,89802.093750
4,Canada,5,7.427,1.32629,0.90563,0.63297,0.32957,0.45811,2015,51354.109375
...,...,...,...,...,...,...,...,...,...,...
775,Malawi,150,3.410,0.19100,0.49500,0.44300,0.08900,0.21800,2019,476.340057
776,Yemen,151,3.380,0.28700,0.46300,0.14300,0.07700,0.10800,2019,658.490784
777,Rwanda,152,3.334,0.35900,0.61400,0.55500,0.41100,0.21700,2019,871.358826
778,Tanzania,153,3.231,0.47600,0.49900,0.41700,0.14700,0.27600,2019,974.751526


In [28]:
WH_df = pd.read_csv("Working_hours.csv", delimiter=",")
WH_df = WH_df.iloc[:, [1, 7, 14]]
#WH_df.columns = ["country", "time", "values"]
WH_df

riigid = happy2["country"].unique().tolist()
WH = WH_df[WH_df['Country'].isin(riigid)]
WH

# Uurime mis välja jäid
riigid2 = WH["Country"].tolist()
valja = set(riigid).difference(set(riigid2))
valja

WH


,Country,Time,Value
0,Australia,2015,1733.0
1,Australia,2016,1740.0
2,Australia,2017,1726.0
3,Australia,2018,1723.0
4,Australia,2019,1712.0
...,...,...,...
184,Costa Rica,2015,2148.2
185,Costa Rica,2016,2204.7
186,Costa Rica,2017,2178.9
187,Costa Rica,2018,2121.0


In [49]:
#Ühendame working hours ja happy andmestikud
happy4
happy4 = happy2[happy2['country'].isin(WH["Country"].tolist())] 
happy4["WH"] = 0
for i in range(0, happy4.shape[0]):
    aasta = happy4.iloc[i, 8]
    riik = happy4.iloc[i, 0]
    #happy4.loc[(WH['country'] == riik) & (WH['time'] == aasta), 'value'] = WH.iloc[riik,'value']
    #happy4.iloc[i, 10] = WH['Value']
happy4


<ipython-input-49-56af0b5c60fb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  happy4["WH"] = 0


ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [4]:
happy_X = happy2.iloc[:, [3,4,5,6,7,9]]
happy_y = happy2.iloc[:, 2]

import statsmodels.api as sm

happy_X = sm.add_constant(happy_X)
model = sm.OLS(happy_y, happy_X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     347.5
Date:                Sun, 13 Dec 2020   Prob (F-statistic):          1.91e-204
Time:                        15:06:34   Log-Likelihood:                -586.23
No. Observations:                 695   AIC:                             1186.
Df Residuals:                     688   BIC:                             1218.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.6926      0.101     26.747      0.0